In [1]:
import dask

if dask.__version__:
    print(f"Dask version {dask.__version__} is installed.")
else:
    print("Dask is not installed. You can install it using 'conda install dask' or 'pip install dask[complete]'.")

Dask version 2022.7.0 is installed.


In [2]:
import pandas as pd
import dask.dataframe as dd
from pathlib import Path
from sqlalchemy import create_engine, text



In [3]:
#Reading in the source file for the project
   
#file_path = r"C:/Desktop/Analysis Projects/LOC Subset.csv"
file_path = r'C:/Desktop/Analysis Projects/Project-3/wdl_data_en_reduced.csv'
# Specifying encoding as'ISO-8859-1' to read file
loc_df = pd.read_csv(file_path, encoding='utf-8')

#Prints the Dask DataFrame structure in which it prints the column headings and datatypes 
print(loc_df)

                                                 wdl_id  \
0                                                     1   
1                                                     2   
2                                                     3   
3                                                     4   
4                                                     5   
...                                                 ...   
23860                                             13569   
23861                                             13570   
23862                                             13571   
23863                                             13572   
23864  >>>>>>> db06b73e751773811f4a728b8efff5b6d1b0332e   

                                                   title  \
0      Antietam, Maryland. Allan Pinkerton, President...   
1      Chola Woman, Full-Length Portrait, Standing, F...   
2               Maps of Ezo, Sakhalin, and Kuril Islands   
3                              Warrior Asahina Koba

C:\Users\acwim\AppData\Local\Temp\ipykernel_17264\501014329.py:6: DtypeWarning: Columns (0,20,33,34,36,37,41,42,43,49,52,54,55,62,63,66,69,70,71,72,74,76,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98) have mixed types. Specify dtype option on import or set low_memory=False.
  loc_df = pd.read_csv(file_path, encoding='utf-8')


In [4]:
# Count of the number of columns and rows in dataframe
column_count = len(loc_df.columns)
row_count = len(loc_df)


# Print results of columns and rows
print("Number of columns:", column_count)
print("Number of records in the DataFrame:", row_count)

Number of columns: 99
Number of records in the DataFrame: 23865


In [5]:
loc_df.columns

Index(['wdl_id', 'title', 'collection', 'description', 'wdl_url',
       'Type of Item', 'Institution', 'Photographer', 'Date Created',
       'Subject Date', 'Place', 'Dewey Decimal Code', 'Additional Subjects',
       'Physical Description', 'Creator', 'Publisher', 'Language', 'Artist',
       'Cartographer', 'Note', 'Delineator', 'Author', 'Surveyor', 'Compiler',
       'Translator', 'Contributor', 'Engraver', 'Cinematographer', 'Director',
       'Illustrator', 'Sponsor', 'Draftsman', 'Editor', 'Corrector',
       'Responsible Party', 'Calligrapher', 'Interviewee', 'Interviewer',
       'Associated Name', 'Printer', 'Lithographer', 'Donor',
       'Recording Engineer', 'Musician', 'Composer', 'Architect', 'References',
       'Attributed Name', 'Dedicatee', 'Signer', 'Copyright Holder',
       'Collector', 'Printmaker', 'Copyright claimant', 'Performer', 'Owner',
       'Commentator', 'Etcher', 'Wood-engraver', 'Scribe',
       'Author of Introduction, etc.', 'Author of Afterword, 

In [6]:
#List of columns to remove
columns_to_remove = ['Artist',
       'Cartographer', 'Note', 'Delineator', 'Author', 'Surveyor', 'Compiler',
       'Translator', 'Contributor', 'Engraver', 'Cinematographer', 'Director',
       'Illustrator', 'Sponsor', 'Draftsman', 'Editor', 'Corrector',
       'Responsible Party', 'Calligrapher', 'Interviewee', 'Interviewer',
       'Associated Name', 'Printer', 'Lithographer', 'Donor',
       'Recording Engineer', 'Musician', 'Composer', 'Architect', 'References',
       'Attributed Name', 'Dedicatee', 'Signer', 'Copyright Holder',
       'Collector', 'Printmaker', 'Copyright claimant', 'Performer', 'Owner',
       'Commentator', 'Etcher', 'Wood-engraver', 'Scribe',
       'Author of Introduction, etc.', 'Author of Afterword, Colophon, etc.',
       'Scientific advisor', 'Speaker', 'Commentator for Written Text',
       'Illuminator', 'Librettist', 'Client', 'Transcriber', 'Collaborator',
       'Producer', 'Videographer', 'Vocalist', 'Patron', 'Funder', 'Annotator',
       'Woodcutter', 'Designer', 'Author in Quotations or Text Extracts',
       'Lyricist', 'Type Designer', 'Animator', 'Writer of added text',
       'Project Director', 'Dubious Author', 'Censor', 'Engineer', 'Lead',
       'Publishing Director', 'Singer', 'Binder', 'Originator', 'Typographer',
       'Book producer', 'Narrator', 'Colorist', 'Arranger', 'Recipient',
       'Correspondent','Photographer']

# Remove only the specified columns
loc2_df = loc_df.drop(columns=columns_to_remove)

In [7]:
# Count of the number of columns and rows in dataframe
column_count = len(loc2_df.columns)

# Print results of columns and rows
print("Number of columns:", column_count)

loc2_df.columns


Number of columns: 16


Index(['wdl_id', 'title', 'collection', 'description', 'wdl_url',
       'Type of Item', 'Institution', 'Date Created', 'Subject Date', 'Place',
       'Dewey Decimal Code', 'Additional Subjects', 'Physical Description',
       'Creator', 'Publisher', 'Language'],
      dtype='object')

In [8]:
#Isolate the Type of Item for the Item_Type Table
#Calculate the number of unique authors in the DataFrame
#item_count = len(loc2_df['Type of Item'].unique())
item_type = (loc2_df['Type of Item'].unique())
print(item_type)

['Prints, Photographs' 'Maps' 'Books' 'Manuscripts' 'Motion Pictures'
 'Sound Recordings' 'Journals' 'Newspapers' nan 'Type of Item']


In [9]:
# Define a dictionary to map item types to codes
type_code_mapping = {
    'Prints, Photographs': 'prph',
    'Maps': 'mp',
    'Books': 'bk',
    'Manuscripts': 'ms',
    'Motion Pictures': 'mp',
    'Sound Recordings': 'sr',
    'Journals': 'jr',
    'Newspapers': 'np'
}

# Add a new column 'Item Code' based on the mapping
loc2_df['Item Code'] = loc2_df['Type of Item'].map(type_code_mapping)

# Print the updated DataFrame
print(loc2_df)


                                                 wdl_id  \
0                                                     1   
1                                                     2   
2                                                     3   
3                                                     4   
4                                                     5   
...                                                 ...   
23860                                             13569   
23861                                             13570   
23862                                             13571   
23863                                             13572   
23864  >>>>>>> db06b73e751773811f4a728b8efff5b6d1b0332e   

                                                   title  \
0      Antietam, Maryland. Allan Pinkerton, President...   
1      Chola Woman, Full-Length Portrait, Standing, F...   
2               Maps of Ezo, Sakhalin, and Kuril Islands   
3                              Warrior Asahina Koba

In [10]:
# Create a DataFrame with unique item codes and their corresponding item types
unique_item_code_df = pd.DataFrame(list(type_code_mapping.items()), columns=['Type of Item', 'Item Code'])

# Export to CSV file
unique_item_code_df.to_csv('unique_item_codes.csv', index=False)



In [11]:
#Isolate the Collection Type for the Collection_Type Table
#Calculate the number of unique authors in the DataFrame
collection_count = len(loc_df['collection'].unique())
collection_type = (loc_df['collection'].unique())
print(collection_count)
print(collection_type)

183
[nan 'Frank and Frances Carpenter Collection'
 'Japanese Prints and Drawings Collection' 'Henry Harrisse Collection'
 'United States History' 'American Imprint Collection'
 'Geographic Card Set of the Russian Empire'
 'Lessing J. Rosenwald Collection'
 'William C. Brumfield Photograph Collection' 'Paper Print Collection'
 'Thereza Christina Maria Collection' 'World War I' 'World History'
 'Great Siberian Railway Guidebooks' 'Thomas Jefferson Library Collection'
 'Chinese Rare Book Collection' 'Pre-1801 Imprint Collection'
 'Hans and Hanni Kraus Sir Francis Drake Collection'
 'Detroit Publishing Company Photograph Collection' 'Naxi Collection'
 'Illuminated Manuscripts from Europe'
 'Views in the Caucasus and Black Sea Area, Russian Empire'
 'World War I Posters'
 'Views of the Napoleonic Campaign Area, Russian Empire'
 'Early Bulgarian Imprint Collection'
 'Legal Acts of the Grand Duchy of Lithuania 1522-1845'
 'Rubbings of Inscriptions on Metal and Stone'
 'George Grantham Bain Co

In [12]:
#Isolate the Collection Type for the Collection_Type Table
#Calculate the number of unique authors in the DataFrame
Institution_count = len(loc_df['Institution'].unique())
Institution_type = (loc_df['Institution'].unique())
print(Institution_count)
print(Institution_type)

121
['Library of Congress' 'National Library of Brazil'
 'Russian State Library' 'National Library of Russia'
 'Mamma Haidara Commemorative Library' 'Bibliotheca Alexandrina'
 'National Library and Archives of Egypt' 'Irish College in Paris'
 'Vilnius University Library' 'National Library of China'
 'Iraqi National Library and Archives' 'National Library of Uganda'
 'University of Texas Libraries' 'National Library of Sweden'
 'Uppsala University Library' 'Boris Yeltsin Presidential Library'
 'National Library of France' 'Tetouan-Asmir Association'
 'National Central Library' 'National Library of Serbia'
 'Qatar National Library' 'Pontifical Institute of Mediaeval Studies'
 'National Library of Israel'
 'Columbus Memorial Library, Organization of American States'
 'University Library in Bratislava'
 'Center for the Study of the History of Mexico CARSO'
 'Royal Netherlands Institute of Southeast Asian and the Caribbean Studies KITLV'
 'U.S. National Archives and Records Administration'


TO SPLIT THE LANGUAGE COLUMN

In [13]:
#Isolate the Language for theLanguage Table
#Calculate the number of unique languages in the DataFrame
language_count = len(loc_df['Language'].unique())
language_type = (loc_df['Language'].unique)
print(language_count)
print(language_type)


207
<bound method Series.unique of 0                   NaN
1                   NaN
2              Japanese
3              Japanese
4               Spanish
              ...      
23860    Persian|Pushto
23861    Persian|Pushto
23862    Persian|Pushto
23863    Persian|Pushto
23864               NaN
Name: Language, Length: 23865, dtype: object>


In [14]:
originallanguage_df=language_type
originallanguage_df



<bound method Series.unique of 0                   NaN
1                   NaN
2              Japanese
3              Japanese
4               Spanish
              ...      
23860    Persian|Pushto
23861    Persian|Pushto
23862    Persian|Pushto
23863    Persian|Pushto
23864               NaN
Name: Language, Length: 23865, dtype: object>

In [15]:
# Extracting the first part and creating a new column
loc2_df['mainlanguage'] = loc2_df['Language'].str.split('|').str[0]
loc2_df

,wdl_id,title,collection,description,wdl_url,Type of Item,Institution,Date Created,Subject Date,Place,Dewey Decimal Code,Additional Subjects,Physical Description,Creator,Publisher,Language,Item Code,mainlanguage
0,1,"Antietam, Maryland. Allan Pinkerton, President...",NaN,"At the outset of the U.S. Civil War, Mathew Br...",https://www.wdl.org/en/item/1,"Prints, Photographs",Library of Congress,1862-10-03,1862-10-03,North America > United States of America > Mar...,973,"Antietam, Battle of, Maryland, 1862|Generals|L...","1 negative : glass, wet collodion",NaN,NaN,NaN,prph,NaN
1,2,"Chola Woman, Full-Length Portrait, Standing, F...",Frank and Frances Carpenter Collection,This photograph of a Bolivian woman is from th...,https://www.wdl.org/en/item/2,"Prints, Photographs",Library of Congress,1911,1900/1923,Latin America and the Caribbean > Bolivia > La...,391,Portrait photographs|Women,NaN,NaN,NaN,NaN,prph,NaN
2,3,"Maps of Ezo, Sakhalin, and Kuril Islands",NaN,This map was made by Fujita Junsai and publish...,https://www.wdl.org/en/item/3,Maps,Library of Congress,1854,1854,East Asia > Japan > Hokkaido|Europe > Russian ...,912,NaN,"1 color map, 115 x 92 centimeters, folded in c...","Fujita, Tonsai",Harimaya Katsugorō,Japanese,mp,Japanese
3,4,Warrior Asahina Kobayashi,Japanese Prints and Drawings Collection,The Japanese art of Ukiyo-e (“Pictures of the ...,https://www.wdl.org/en/item/4,"Prints, Photographs",Library of Congress,1862,1862,East Asia > Japan,769|792,Actors|Costumes|Japanese drama|Portrait prints...,"1 print: woodcut, color ; 29.0 x 21.3 centimeters",NaN,NaN,Japanese,prph,Japanese
4,5,"Manuscript Map of Dagua River Region, Colombia",NaN,This beautiful pen-and-ink and watercolor map ...,https://www.wdl.org/en/item/5,Maps,Library of Congress,1764,1764,Latin America and the Caribbean > Colombia > V...,912,Rivers,1 manuscript map : color ; 60 x 86 centimeters,NaN,NaN,Spanish,mp,Spanish
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23860,13569,"Zhvandūn, Number 27, Saturday, September 22, 1973",Zhvandūn Magazine,"Zhvandūn, generally known as “Zhwandun,” was o...",https://www.wdl.org/en/item/13569,Journals,Library of Congress,9/22/1973,9/22/1973,Central and South Asia > Afghanistan,079|958,Library of Congress Afghanistan Project|Manner...,35-37 centimeters,NaN,Ministry of Information and Culture and Union ...,Persian|Pushto,jr,Persian
23861,13570,"Zhvandūn, Numbers 28-29, Saturday, October 6, ...",Zhvandūn Magazine,"Zhvandūn, generally known as “Zhwandun,” was o...",https://www.wdl.org/en/item/13570,Journals,Library of Congress,10/6/1973,10/6/1973,Central and South Asia > Afghanistan,079|958,Library of Congress Afghanistan Project|Manner...,35-37 centimeters,NaN,Ministry of Information and Culture and Union ...,Persian|Pushto,jr,Persian
23862,13571,"Zhvandūn, Number 30, Saturday, October 13, 1973",Zhvandūn Magazine,"Zhvandūn, generally known as “Zhwandun,” was o...",https://www.wdl.org/en/item/13571,Journals,Library of Congress,10/13/1973,10/13/1973,Central and South Asia > Afghanistan,079|958,Library of Congress Afghanistan Project|Manner...,35-37 centimeters,NaN,Ministry of Information and Culture and Union ...,Persian|Pushto,jr,Persian
23863,13572,"Zhvandūn, Number 31, Saturday, October 20, 1973",Zhvandūn Magazine,"Zhvandūn, generally known as “Zhwandun,” was o...",https://www.wdl.org/en/item/13572,Journals,Library of Congress,10/20/1973,10/20/1973,Central and South Asia > Afghanistan,079|958,"Israel-Arab War, 1973|Library of Congress Afgh...",35-37 centimeters,NaN,Ministry of Information and Culture and Union ...,Persian|Pushto,jr,Persian


TO SPLIT PUBLISHER

In [16]:
#Isolate the Collection Type for the Collection_Type Table
#Calculate the number of unique authors in the DataFrame
Publisher_count = len(loc_df['Publisher'].unique())
Publisher_type = (loc_df['Publisher'].unique())
print(Publisher_count)
print(Publisher_type)

1129
[nan 'Harimaya Katsugorō' 'Dutch West India Company' ... 'Zhang Dingsi'
 'Ministry of Information and Culture and Union of the Writers of the Democratic Republic of Afghanistan'
 'Publisher']


Using Pandas to test creating tables and connecting to Postgresql

In [17]:

# pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [18]:

import psycopg2
from psycopg2 import sql

In [25]:
conn = psycopg2.connect(
    dbname="EngineerDB",
    user="postgres",
    password="password",
    host="localhost",
    port="5433"
)

cursor = conn.cursor()